In [ ]:
%pylab notebook
import matplotlib.pyplot as plt
from charistools.meltModels import TriSurfTempIndexMelt
from charistools.meltModels import ImshowTriSurfMelt
from charistools.meltModels import PlotTriSurfInput
from charistools.meltModels import PlotTriSurfMelt
from charistools.modelEnv import ModelEnv
import pandas as pd
import re
import os

In [ ]:
configFile = '/Users/brodzik/projects/CHARIS/charistools_test_data/calibration_modelEnv_config.ini'
myEnv = ModelEnv(tileConfigFile=configFile)

In [ ]:
#drainageID = 'IN_Hunza_at_DainyorBridge'
#drainageID = 'AM_Vakhsh_at_Komsomolabad'
#drainageID = 'GA_Karnali_at_Benighat'
#drainageID = 'GA_Narayani_at_Devghat'
drainageID = 'GA_SaptaKosi_at_Chatara'
%cd ~/projects/CHARIS/charistools_test_data/derived_hypsometries/$drainageID
%pwd

In [ ]:
ablation_method = 'albedo_mod10a1'
threshold = 0.40
year = 2002
year_str = str(year)
input = myEnv.model_inputs(drainageID=drainageID,
                           year=year,
                           ablation_method=ablation_method,
                           threshold=threshold)

In [ ]:
#input['exposed_glacier_ice_by_elevation_hyps'].data

In [ ]:
if drainageID == "IN_Hunza_at_DainyorBridge":
    best_model = "8.0_9.44_8.0_15.0"
    best_model = "8.89_9.0_12.0_19.0"
elif drainageID == "AM_Vakhsh_at_Komsomolabad":
    best_model = "1.78_2.33_4.0_13.0"
elif drainageID == "GA_Karnali_at_Benighat":
    best_model = "4.0_5.0_0.44_7.0"
elif drainageID == "GA_Narayani_at_Devghat":
    best_model = "4.0_5.0_4.0_7.0"
elif drainageID == "GA_SaptaKosi_at_Chatara":
    best_model = "4.0_5.0_4.0_7.0"    
    
    
(min_snow_ddf, max_snow_ddf, min_ice_ddf, max_ice_ddf) = best_model.split("_")
min_snow_ddf = float(min_snow_ddf)
max_snow_ddf = float(max_snow_ddf)
min_ice_ddf = float(min_ice_ddf)
max_ice_ddf = float(max_ice_ddf)
print(min_snow_ddf, max_snow_ddf, min_ice_ddf, max_ice_ddf)

In [ ]:
(SOLmelt, SOImelt, EGImelt) = TriSurfTempIndexMelt(
    input['snow_on_land_by_elevation_filename'],
    input['snow_on_ice_by_elevation_filename'],
    input['exposed_glacier_ice_by_elevation_filename'],
    input['temperature_by_elevation_filename'],
    min_snow_ddf=min_snow_ddf,
    max_snow_ddf=max_snow_ddf,
    min_ice_ddf=min_ice_ddf,
    max_ice_ddf=max_ice_ddf)

In [ ]:
import os
import re
p = re.compile(drainageID)
outdir = '/Users/brodzik/projects/CHARIS/charistools_test_data/derived_hypsometries/' + drainageID
SOLmeltfile = os.path.basename(input['snow_on_land_by_elevation_filename'])
SOLmeltfile = SOLmeltfile.replace('_area_by_elev', '_melt_by_elev')
SOLmeltfile = outdir + '/' + p.sub(drainageID + '.best_model', SOLmeltfile)

SOImeltfile = os.path.basename(input['snow_on_ice_by_elevation_filename'])
SOImeltfile = SOImeltfile.replace('_area_by_elev', '_melt_by_elev')
SOImeltfile = outdir + '/' + p.sub(drainageID + '.best_model', SOImeltfile)

EGImeltfile = os.path.basename(input['exposed_glacier_ice_by_elevation_filename'])
EGImeltfile = EGImeltfile.replace('_area_by_elev', '_melt_by_elev')
EGImeltfile = outdir + '/' + p.sub(drainageID + '.best_model', EGImeltfile)

print(SOLmeltfile)
print(SOImeltfile)
print(EGImeltfile)

In [ ]:
columns = [float(i) for i in SOLmelt.data.columns]
SOLmelt.data.columns = columns
SOLmelt.data = SOLmelt.data[sort(columns)]

columns = [float(i) for i in SOImelt.data.columns]
SOImelt.data.columns = columns
SOImelt.data = SOImelt.data[sort(columns)]

columns = [float(i) for i in EGImelt.data.columns]
EGImelt.data.columns = columns
EGImelt.data = EGImelt.data[sort(columns)]


In [ ]:
from charistools.hypsometry import Hypsometry
SOLmelt.write(filename=SOLmeltfile, decimal_places=2, verbose=True)
SOImelt.write(filename=SOImeltfile, decimal_places=2, verbose=True)
EGImelt.write(filename=EGImeltfile, decimal_places=2, verbose=True)

In [ ]:
outfile = SOImeltfile
p = re.compile(r'snow_on_ice.+')
outfile = p.sub('melt_tseries.png', outfile)
print(outfile)
outhypsfile = SOImeltfile
p = re.compile(r'snow_on_ice.+')
outhypsfile = p.sub('melt_hyps.png', outhypsfile)
print(outhypsfile)

In [ ]:
print(np.nanmin(SOLmelt.data.values),np.nanmax(SOLmelt.data.values))
print(np.amin(EGImelt.data.values),np.amax(EGImelt.data.values))

In [ ]:
# Temporary fix for SOLmelt.data NaNs
SOLmelt.data = SOLmelt.data.fillna(value=0.)

In [ ]:
fig, axes = plt.subplots(3,1)
ImshowTriSurfMelt(axes, SOLmelt, SOImelt, EGImelt)
#axes[2] = SOLmelt.imshow(ax=axes[2], title='Snow-on-Land Melt',
#                         cmap="Greens",
#                         vmin=np.nanmin(SOLmelt.data.values),
#                         vmax=np.nanmax(SOLmelt.data.values))
for ax in axes:
    ax.set_title(drainageID + " (" + year_str + ") " + ax.get_title())
fig.tight_layout()
fig.savefig(outhypsfile)

In [ ]:
rainfallFile = myEnv.calibration_filename(type="rainfall", 
                                          drainageID=drainageID)
runoffFile = myEnv.calibration_filename(type="runoff", 
                                          drainageID=drainageID)
print(rainfallFile)
print(runoffFile)

In [ ]:
from charistools.timeSeries import TimeSeries
rainfall = TimeSeries(rainfallFile)
runoff = TimeSeries(runoffFile)

In [ ]:
monthly_rainfall = rainfall.data['rainfall'][year_str + '-01-01':year_str + '-12-01']
monthly_runoff = runoff.data['runoff'][year_str + '-01-01':year_str + '-12-01']
print(monthly_rainfall)
print(monthly_runoff)

In [ ]:
melt_by_doy = (                                                             
        SOLmelt.data_by_doy() +                                           
        SOImelt.data_by_doy() +                                           
        EGImelt.data_by_doy())
total_melt = melt_by_doy.sum()
print("total melt = %.2f" % total_melt)
melt_by_month = melt_by_doy.groupby([pd.TimeGrouper('M')]).sum().to_frame(name='melt')
# print(melt_by_month)
yyyymm_index = pd.to_datetime(melt_by_month.index.map(lambda x: x.strftime('%Y-%m-15')))
df = pd.DataFrame(data=melt_by_month.values, index=yyyymm_index, columns=['melt'])
print(df)
print(monthly_rainfall.values)
df["rainfall"] = monthly_rainfall.values
df["runoff"] = monthly_runoff.values
df["melt+rainfall"] = df["melt"] + df["rainfall"]
print(df)
np.amax(df["melt+rainfall"])

In [ ]:
fig, ax = plt.subplots(3,1,figsize=(9,9))
ax[0] = PlotTriSurfInput(ax[0], 
                         input['snow_on_land_by_elevation_hyps'],
                         input['snow_on_ice_by_elevation_hyps'],
                         input['exposed_glacier_ice_by_elevation_hyps'],
                         input['temperature_by_elevation_hyps'],
                         title="Inputs for %s (%d)" % (drainageID, year))
ax[1] = PlotTriSurfMelt(ax[1], SOLmelt, SOImelt, EGImelt, 
                     title="Modelled melt for %s (%d)" % (drainageID, year))
ax[1].text(ax[1].get_xlim()[0] + (0.03 * (ax[1].get_xlim()[1] - ax[1].get_xlim()[0])), 
           ax[1].get_ylim()[1] * 0.7,
           'snow DDF = %.2f - %.2f $mm/C/day$\nice DDF = %.2f - %.2f $mm/C/day$' % 
           (min_snow_ddf, max_snow_ddf, min_ice_ddf, max_ice_ddf),
           style='italic',
            bbox={'facecolor':'gray', 'alpha':0.1, 'pad':10})
df[['runoff']].plot(ax=ax[2], kind="bar", color=(0.8, 0.8, 0.8))
df[['rainfall','melt']].plot(ax=ax[2], kind="bar", position=0., 
                             stacked=True, color=['c','b'])
ax[2].set_title("Monthly Runoff vs. (Rainfall + Melt)")
ax[2].set_ylabel('Volume (' + r'$km^3$' + ')') 
ax[2].xaxis.set_major_formatter(plt.NullFormatter())
for container in ax[2].containers:
    plt.setp(container, width=0.25)
ax[2].set_ylim([0,1.1 * np.amax(df["melt+rainfall"])])
if df["runoff"].sum() > 0.:
    total_runoff = df["runoff"].sum()
else:
    total_runoff = np.nan
ax[2].text(ax[2].get_xlim()[0] + (0.03 * (ax[2].get_xlim()[1] - ax[2].get_xlim()[0])), 
           ax[2].get_ylim()[1] * 0.7,
           'melt + rainfall = %.2f $km^2$\nrunoff = %.2f $km^2$' % 
           (df["melt+rainfall"].sum(), total_runoff),
           style='italic',
            bbox={'facecolor':'gray', 'alpha':0.1, 'pad':10})
handles, labels = ax[2].get_legend_handles_labels()
ax[2].legend(reversed(handles), reversed(labels))

fig.tight_layout()
fig.savefig(outfile)


In [ ]:
outfile